In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

from src.datasetgenerator import StratzQuery, DatasetGenerator

In [2]:
load_dotenv()
#querier = StratzQuery(os.getenv('STRATZ_TOKEN'))
#querier.get_match(7590822094)

csv_path = os.getenv('CSV_PATH')

if not os.getenv('STRATZ_TOKEN'):
    raise FileNotFoundError("Not found STRATZ_TOKEN on .env")

generator = DatasetGenerator(os.getenv('STRATZ_TOKEN'), csv_path)

In [3]:

league = generator.get_professional_league(16842)
matches = league['matches']
len(matches)

43

In [4]:
matches[0].keys()

dict_keys(['id', 'didRadiantWin', 'durationSeconds', 'startDateTime', 'firstBloodTime', 'averageRank', 'players', 'playbackData', 'averageImp', 'actualRank', 'radiantKills', 'direKills', 'radiantTeam', 'direTeam'])

In [5]:
matches[0]['playbackData']['wardEvents']

[{'indexId': 450,
  'time': -63,
  'positionX': 126,
  'positionY': 126,
  'fromPlayer': 4,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 454,
  'time': -62,
  'positionX': 154,
  'positionY': 100,
  'fromPlayer': 2,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 459,
  'time': -53,
  'positionX': 120,
  'positionY': 122,
  'fromPlayer': 128,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 517,
  'time': -17,
  'positionX': 138,
  'positionY': 82,
  'fromPlayer': 129,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 548,
  'time': -9,
  'positionX': 128,
  'positionY': 74,
  'fromPlayer': 130,
  'wardType': 'SENTRY',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 854,
  'time': 14,
  'positionX': 168,
  'positionY': 92,
  'fromPlayer': 2,
  'wardType': 'SENTRY',
  'action': 'SPAWN',
  'playerDestroyed': None},
 

In [6]:
def get_deaths_through_ward(match:dict):

    # if match[']
    deaths_through_ward = {
        'time': [],
        'isPlayerRadiant': [],
        'positionX': [],
        'positionY': [],
    }
    for player in match["players"]:
        isPlayerRadiant = player['isRadiant']
        deaths_events = player['playbackData']['deathEvents']
        for d_e in deaths_events:
            if not d_e['isWardWalkThrough']:
                continue
            deaths_through_ward['time'].append(d_e['time'])
            deaths_through_ward['isPlayerRadiant'].append(isPlayerRadiant)
            deaths_through_ward['positionX'].append(d_e['positionX'])
            deaths_through_ward['positionY'].append(d_e['positionY'])
    
    df_deaths = pd.DataFrame.from_dict(deaths_through_ward)
    df_deaths['match'] = match['id']
    return df_deaths

def get_deaths_through_ward_league(league_id):
    
    league = generator.get_professional_league(league_id=league_id)
    matches = league['matches']

    df_deaths = pd.concat([get_deaths_through_ward(match) for match in matches])
    return df_deaths

df_deaths = get_deaths_through_ward_league(16842)
df_deaths

,time,isPlayerRadiant,positionX,positionY,match
0,1494,True,182,60,7811671022
1,1418,True,158,70,7811671022
2,1512,True,174,64,7811671022
3,829,True,152,64,7811671022
4,1487,True,194,70,7811671022
...,...,...,...,...,...
27,479,False,174,96,7803951479
28,786,False,170,92,7803951479
29,1363,False,150,148,7803951479
30,1781,False,66,192,7803951479


In [7]:
from collections import defaultdict

def get_match_wards(match:dict) -> dict:
    player_team_by_slot = {player['playerSlot']: player['isRadiant'] for player in match['players']}
    match_id = match['id']
    ward_events = match['playbackData']['wardEvents']
    wards = defaultdict(lambda: {})
    for w_e in ward_events:
        if w_e["action"] == "SPAWN":
            wards[w_e['indexId']] = {
                "id": f"{match_id}_{w_e['indexId']}",
                "match": match_id,
                "spawned_time": w_e["time"],
                "despawned_time": None,
                "positionX": w_e["positionX"],
                "positionY": w_e["positionY"],
                "wardType": w_e["wardType"],
                "isRadiant": player_team_by_slot[w_e["fromPlayer"]],
                "playerDestroyed": w_e["playerDestroyed"],
            }
            continue
        wards[w_e['indexId']]["despawned_time"] = w_e['time']
    

    # get kills around
    
    return dict(wards)

wards = get_match_wards(matches[0])
wards.keys(), len(wards.keys())

(dict_keys([450, 454, 459, 517, 548, 854, 1877, 1884, 2217, 2224, 2491, 2257, 881, 2227, 2383, 2400, 1656, 2215, 1950, 1683, 1126, 2487, 2146, 2457, 2405, 2047, 2425, 2451, 2088, 1040, 2551, 814, 2226, 2177, 2128, 1918, 2480, 2209, 856, 1848, 920, 1598, 747, 2508, 1720, 2529, 857, 1862, 1554, 2090, 2422, 1672, 541, 921, 1054, 2361, 2397, 1743, 2554, 866, 228, 261, 230, 2132, 2447, 2170, 746, 2152, 848, 2049, 1773, 2038, 2338, 1925, 1858, 1599, 1566, 1810, 2396, 2331, 2079, 2534, 1959, 231, 1926, 2134, 2308, 1768, 2273, 2406, 879, 1631, 452, 461, 775, 1624, 1668, 1713, 1736, 1876, 1775]),
 101)

In [8]:
from collections import defaultdict

def map_death_count(df_wards, df_deaths):
    death_time = df_deaths['time'].item()
    return (df_wards['spawned_time'] < death_time) & (death_time > df_wards['despawned_time'])

def get_df_match_wards(match:dict) -> dict:
    if not match['playbackData']:
        print(f"warning: no playbackData found for match {match['id']}")
        return
    wards = get_match_wards(match)
    df_wards = defaultdict(lambda: [])
    for id, w in wards.items():
        for key, item in w.items():
            df_wards[key].append(item)
    df_wards = pd.DataFrame.from_dict(df_wards).set_index("id")

    none_despawned_mask = df_wards['despawned_time'].isna()
    df_wards.loc[none_despawned_mask, 'despawned_time'] = match['durationSeconds']

    # df_deaths = get_deaths_through_ward(match)
    
    # df_wards['possible_enemies_death'] = 0
    # return map_death_count(df_wards, df_deaths)

    
    # df_wards['possible_enemies_death'] = 0


    df_wards['spawned_time_minute'] = (df_wards['spawned_time'] // 60).astype(int)
    df_wards['despawned_time_minute'] = (df_wards['despawned_time'] // 60).astype(int)
    
    df_wards['radiantTeam'] = match['radiantTeam']['name']
    df_wards['direTeam'] = match['direTeam']['name']


    return df_wards

def get_league_df_wards(league_id):
    league = generator.get_professional_league(league_id)
    matches = league['matches']
    ward_matches_df = [get_df_match_wards(match) for match in matches]
    df_wards = pd.concat(ward_matches_df)
    df_wards['league'] = league_id
    df_wards['region'] = league['region']
    return df_wards

df_wards = get_league_df_wards(16842)
df_wards.isna().any()

match                    False
spawned_time             False
despawned_time           False
positionX                False
positionY                False
wardType                 False
isRadiant                False
playerDestroyed           True
spawned_time_minute      False
despawned_time_minute    False
radiantTeam              False
direTeam                 False
league                   False
region                   False
dtype: bool

In [9]:
df_wards

,match,spawned_time,despawned_time,positionX,positionY,wardType,isRadiant,playerDestroyed,spawned_time_minute,despawned_time_minute,radiantTeam,direTeam,league,region
id,,,,,,,,,,,,,,
7811671022_450,7811671022,-63,296.0,126,126,OBSERVER,True,None,-2,4,Tundra Esports,PSG.Quest,16842,EUROPE
7811671022_454,7811671022,-62,297.0,154,100,OBSERVER,True,None,-2,4,Tundra Esports,PSG.Quest,16842,EUROPE
7811671022_459,7811671022,-53,306.0,120,122,OBSERVER,False,None,-1,5,Tundra Esports,PSG.Quest,16842,EUROPE
7811671022_517,7811671022,-17,342.0,138,82,OBSERVER,False,None,-1,5,Tundra Esports,PSG.Quest,16842,EUROPE
7811671022_548,7811671022,-9,15.0,128,74,SENTRY,False,None,-1,0,Tundra Esports,PSG.Quest,16842,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7803951479_2959,7803951479,2897,2955.0,136,138,SENTRY,False,None,48,49,Tundra Esports,NAVI Junior,16842,EUROPE
7803951479_2072,7803951479,2943,3080.0,182,140,OBSERVER,False,None,49,51,Tundra Esports,NAVI Junior,16842,EUROPE
7803951479_2972,7803951479,2956,3080.0,174,152,SENTRY,False,None,49,51,Tundra Esports,NAVI Junior,16842,EUROPE


In [10]:
df_wards[['positionX', 'positionY']].describe()

,positionX,positionY
count,4112.000000,4112.000000
mean,125.911965,124.170720
std,30.169634,32.184294
min,58.000000,58.000000
25%,100.000000,96.000000
50%,124.000000,122.000000
75%,150.000000,152.000000
max,194.000000,196.000000


In [11]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# bg_img = np.array(Image.open('assets\Game_map_7.33.webp').resize((500,500)))
# # bg_img = mpimg.imread('assets\Game_map_7.33.webp')
# print(type(bg_img), bg_img.shape)
# for name, group in df_wards.groupby('match'):

#     print(f"\n{group['match'].unique()} - {group['radiantTeam'].unique()} x {group['direTeam'].unique()}")
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ax.imshow(bg_img, extent=[55, 200, 55, 200])  # Adjust extent to match your data range
#     ax.scatter(group['positionX'], group['positionY'], label=f'Match {name}', color=group['wardType'].map(lambda x: "yellow" if x == "OBSERVER" else "blue"))

#     # Show plot
#     plt.show()


In [12]:
df_wards.count().max(), df_deaths.count().max()

(np.int64(4112), np.int64(1189))